### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [ ]:
step 1: 

In [ ]:
minimize communication
maximize storage
minimize FLOPs

In [5]:
class _P2P:
    def _recv_metadata(self, src_rank, parallel_context, parallel_mode):
        process_group = parallel_context.get_group(parallel_mode)
        
        dtype = torch.tensor(0)
        dist.recv(dtype, src=src_rank, group=process_group)
        dtype = ID_TO_DYPE[dtype]
        
        requires_grad = torch.tensor(0)
        dist.recv(requires_grad, src=src_rank, group=process_group)
        requires_grad = True if requires_grad == 1 else False
        
        shape = torch.tensor(0)
        dist.recv(shape, src=src_rank, group=process_group)
        return dtype, requires_grad, shape
    
    def recv(self, src_rank, parallel_context, parallel_mode):
        process_group = parallel_context.get_group(parallel_mode)
        dtype, requires_grad, shape = self._recv_metadata(src_rank, parallel_context)
        
        data = torch.zeros(shape, requires_grad=requires_grad, dtype=dtype)
        dist.recv(data, src=src_rank, process_group)

In [6]:
def recv(src_rank, dst_rank, parallel_context, parallel_mode = ParallelMode.PIPELINE):
    rank = parallel_context.get_local_rank(parallel_mode)
    if rank == dst_rank:
        return _P2P().recv(src_rank, parallel_context, parallel_mode)

In [7]:
import os

In [ ]:
class MPU:
    def __init__(self, master_addr, master_port, backend):
        if not dist.is_initialized():
            init_method = f"tcp://{master_addr}:{master_port}"
            rank = os.getenv("RANK")
            world_size = os.getenv("WORLD_SIZE")
            
            self.set_device(rank)
            
            dist.init_process_group(
                rank=rank,
                world_size=world_size,
                init_method=init_method,
                backend=backend
            )
    
    def set_device(self, rank):
        n_devices = torch.cuda.device_count()
        
        if n_devices > 1:
            torch.cuda.set_device(rank%n_devices)

In [ ]:
step 1: mask targets
step 2: calcualte the local predicted logits
step 3: calculate the global predicted logits
step 4: sum
step 5: 

In [ ]:
using namespace std;

In [ ]:
int main() {
    int x = 1;
    std::cout << &x;
}

In [ ]:
pod > node > container

In [ ]:
recompute, forward, backward

In [ ]:
class Recompute(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        ctx.function = function
        ctx.input = input
        
        return phony
    
    @staticmethod
    def backward(ctx, grad_input):
        with torch.enable_grad():
            output = ctx.function(ctx.input)
        
        ctx.recomputed.append((output, input))

In [ ]:
node changes
keep track

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
embed = model.embed(tokens)

In [9]:
def by_row_parallelism(inputs, weights):
    inp_per_partition = inputs.shape[-1] // 2
    w_per_partition = weights.shape[-1] // 2
    
    inp1 = inputs[:, inp_per_partition:]
    inp2 = inputs[:, :inp_per_partition]
    w1 = weights[:w_per_partition, :]
    w2 = weights[w_per_partition:, :]
    
    out1 = inp1 @ w1
    out2 = inp2 @ w2
    
    return out1 + out2

In [10]:
world_size = 16

In [11]:
tensor_parallel_size = 2

In [12]:
pipeline_parallel_size = 4

In [13]:
num_pipeline_parallel_groups = world_size // pipeline_parallel_size

In [14]:
data_parallel_groups = []

In [15]:
for i in range(pipeline_parallel_size):
    start_rank = i*num_pipeline_parallel_groups
    end_rank = (i+1)*num_pipeline_parallel_groups
    
    for j in range(tensor_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
start_idx + partition_size

In [17]:
from typing import cast, List

In [ ]:
numbers = cast(List[int], numbers)

In [18]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
job selector > spawn workers > pool monitor

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
ln02 = model.blocks[0].ln2
mlp0 = model.blocks[0].mlp0

In [ ]:
torch.argmin()

In [ ]:
 ep_info = [
    {"enc": dict(), "dec": dict()}
    for i in range(len(ExpertParallelGroupInitializer.BASE_ELEMENT))
]

# Encoder
# if "enc" in self.expert_parallel_size:
#     self.construct_stack_parallel_info(ep_info, stack="enc")

# Decoder
if "dec" in self.expert_parallel_size:
    self.construct_stack_parallel_info(ep_info, stack="dec")

ep_info += [ParallelMode.EXPERT]

In [ ]:
for k, v in self.expert_parallel_size[stack].items():
    cur_ep_info = self.init_dist_group_per_unit(v)

    for i, info in enumerate(cur_ep_info):
        ep_info[i][stack][k] = info

return

In [20]:
from dataclasses import dataclass

import torch

In [21]:
@dataclass
class Metadata:
    idx: int

In [41]:
class Back(torch.autograd.Function):
    @staticmethod
    def forward(ctx, metadata, input):
        ctx.metadata = metadata
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return ctx.metadata

In [42]:
tensor = torch.randn(2, 4)

In [43]:
metadata = Metadata(idx=1)

In [44]:
Back.apply(metadata, tensor).sum().backward()

AttributeError: attribute 'metadata' of 'torch._C._FunctionBase' objects is not writable

In [45]:
def function(self):
    return self.x

In [48]:
class Job:
    def __init__(self, function):
        self.function = function
        self.x = 1
    
    def run_func(self):
        return self.function()

In [49]:
job = Job(function)

In [50]:
job.run_func()

TypeError: function() missing 1 required positional argument: 'self'